**Academy of Py**

In [1]:
import pandas as pd

In [17]:
path1 = 'Resources/schools_complete.csv'
schools = pd.read_csv(path1, encoding='iso-8859-1', low_memory=False)
schools.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [18]:
path2 = 'Resources/students_complete.csv'
students = pd.read_csv(path2, encoding='iso-8859-1', low_memory=False)
students.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


**District Summary**

In [60]:
total_schools = len(schools['School ID'].unique())
total_students = len(students['Student ID'].unique())
total_budget = schools['budget'].sum()

av_math_score = round((students['math_score'].mean()),2)
av_read_score = round((students['reading_score'].mean()),2)

math_pass_df = students.loc[students['math_score'] >= 70]
math_pass = round((len(math_pass_df) / total_students * 100),2)
read_pass_df = students.loc[students['reading_score'] >= 70]
read_pass = round((len(read_pass_df) / total_students * 100),2)
ov_pass_df = pd.merge(math_pass_df, read_pass_df, on='Student ID')
ov_pass = round((len(ov_pass_df) / total_students * 100),2)
distr_df = pd.DataFrame({'Total Schools': [total_schools], 'Total Students': [total_students], 'Total Budget':
                         [total_budget], 'Average Math Score': [av_math_score], 'Average Reading Score':
                         [av_read_score], '% Passing Math': [math_pass], '% Passing Reading': [read_pass],
                         '% Overall Passing Rate': [ov_pass]})
distr_df['Total Budget'] = distr_df['Total Budget'].map("$ {:,.2f}".format)
distr_df['Total Students'] = distr_df['Total Students'].map(" {:,.2f}".format)
distr_df_answ = distr_df[['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score',
                         '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
distr_df_answ

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170.00","$ 24,649,428.00",78.99,81.88,74.98,85.81,65.17


**School Summary**

In [112]:
school_sum = schools[['name', 'type', 'size', 'budget']]
school_sum = school_sum.assign(Per_Student_Budget = (school_sum['budget'] / school_sum['size']))
school_sum.columns = ['School name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget']

students_extr = students[['school', 'reading_score', 'math_score']]
stud_sum = students_extr.groupby(['school'])
stud_sum_df = pd.DataFrame(stud_sum.mean())
stud_sum_df.reset_index(inplace=True)
stud_sum_df.columns = ['School name', 'Average Reading Score', 'Average Math Score']

students_extr_math = students[['school', 'math_score']]
math_all_df = students_extr_math.loc[students_extr_math['math_score'] >= 70]
math_sum = math_all_df.groupby(['school'])
math_sum_df = pd.DataFrame(math_sum.count())
math_sum_df.reset_index(inplace=True)
math_sum_df.columns = ['School name', 'Passed Math Count']

students_extr_read = students[['school', 'reading_score']]
read_all_df = students_extr_read.loc[students_extr_read['reading_score'] >= 70]
read_sum = read_all_df.groupby(['school'])
read_sum_df = pd.DataFrame(read_sum.count())
read_sum_df.reset_index(inplace=True)
read_sum_df.columns = ['School name', 'Passed Read Count']
#read_sum_df['% Passing Math'] = read_sum_df['Passed Read Count'] / school_sum['Total Students']*100


new_sum1 = pd.merge(school_sum, stud_sum_df, on='School name')
new_sum2 = pd.merge(new_sum1, math_sum_df, on='School name')
new_sum = pd.merge(new_sum2, read_sum_df, on='School name')
new_sum['% Passing Math'] = round((new_sum['Passed Math Count'] / new_sum['Total Students'] * 100),2)
new_sum['% Passing Reading'] = round((new_sum['Passed Read Count'] / new_sum['Total Students'] * 100),2)
new_sum['% Overall Passing Rate'] = (new_sum['% Passing Math'] + new_sum['% Passing Reading'])/2
#new_summ = new_sum[['School name', 'Total Students', '% Passing Math']]
distr_df_new = new_sum[['School name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                     'Average Reading Score', 'Average Math Score', '% Passing Math', '% Passing Reading',
                     '% Overall Passing Rate']]
distr_answ = distr_df_new.set_index('School name')
distr_answ

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,81.182722,76.629414,65.68,81.32,73.500
Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,65.99,80.74,73.365
Shelton High School,Charter,1761,1056600,600.0,83.725724,83.359455,93.87,95.85,94.860
Hernandez High School,District,4635,3022020,652.0,80.934412,77.289752,66.75,80.86,73.805
Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,93.39,97.14,95.265
Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,93.87,96.54,95.205
Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,94.13,97.04,95.585
Bailey High School,District,4976,3124928,628.0,81.033963,77.048432,66.68,81.93,74.305
Holden High School,Charter,427,248087,581.0,83.814988,83.803279,92.51,96.25,94.380
